# As ações mais baratas da B3 pela Fórmula de Benjamin Graham

## 1. Objetivo

- Desenvolver um script em Python utilizando a biblioteca Selenium para realizar o web scraping do site [Fundamentus](https://www.fundamentus.com.br/) com o propósito de coletar dados dos ativos listados na B3. Após a coleta, os dados serão tratados e utilizados para identificar os ativos mais baratos da bolsa com base na Fórmula de Benjamin Graham, que leva em consideração métricas como Lucro por Ação (LPA) e Valor Patrimonial por Ação (VPA).


- Vamos combinar o Indicador EV/EBIT e o Valor Intrínseco pela Fórmula de Graham para avaliar a atratividade de um ativo na Bolsa de Valores.


- Um EV/EBIT baixo geralmente indica que a empresa está sendo negociada a um preço atrativo em relação ao seu lucro operacional, sugerindo potencial de valorização.


- Para garantir uma análise conservadora e proteger contra incertezas, aplicaremos margens de segurança de 60%, 50% e 40% ao valor intrínseco calculado.


- A decisão de compra será tomada apenas se o preço de mercado da ação estiver abaixo do valor ajustado pela margem de segurança escolhida. Entretanto, esta não deve ser considerada uma recomendação de compra ou venda de qualquer ativo.

## 2. Coleta de Dados

- Utilizaremos o Selenium em conjunto com o navegador Google Chrome para realizar o web scraping.


- Para a instalar a biblioteca do Selenium abra o prompt do anaconda e execute o comando:
```
pip install --upgrade selenium
```

- O Selenium para funcionar, precisa também do webdriver que no Chrome se chama chromedriver. O webdriver é o que vai permitir o Selenium comandar o navegador.


- O webdriver-manager automatiza a atualização do ChromeDriver, eliminando a necessidade de baixá-lo toda vez que o Google Chrome for atualizado. 
```
pip install webdriver-manager
``` 

### 2.1 Instalação e Configurações Importantes do Selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)

In [3]:
# Para acessar ao site  Fundamentus com o Selenium, usaremos a variável navegador que foi configurada
navegador.get("https://www.fundamentus.com.br/")

### 2.2 Filtrando Ações por P/L e Liquidez com Métodos de Seleção do Selenium

- Como não queremos empresas com lucro negativo, definiremos o P/L (Preço/Lucro) mínimo como 0 e a liquidez diária mínima como R$ 1.000.000, sendo que este último critério pode ser ajustado conforme necessário.

In [4]:
from selenium.webdriver.common.by import By

navegador.find_element(By.XPATH, '/html/body/div[1]/div[1]/div[1]/span/a[1]').click()

navegador.find_element(By.XPATH, '/html/body/div[1]/div[2]/form/table/tbody/tr[1]/td[2]/span/input[1]').click()
navegador.find_element(By.XPATH, '/html/body/div[1]/div[2]/form/table/tbody/tr[1]/td[2]/span/input[1]').send_keys("0")
navegador.find_element(By.XPATH, '/html/body/div[1]/div[2]/form/table/tbody/tr[16]/td[2]/span/input[1]').send_keys("1000000")

navegador.find_element(By.XPATH, '/html/body/div[1]/div[2]/form/input').click()

### 2.3 Extraindo as informações  da tabela

In [5]:
# Localizando o elemento tabela pelo ID
table = navegador.find_element(By.ID, 'resultado')

# Identificando todas as linhas da tabela
rows = table.find_elements(By.TAG_NAME, 'tr')

In [6]:
rows[0].text

'Papel Cotação P/L P/VP PSR Div.Yield P/Ativo P/Cap.Giro P/EBIT P/Ativ Circ.Liq EV/EBIT EV/EBITDA Mrg Ebit Mrg. Líq. Liq. Corr. ROIC ROE Liq.2meses Patrim. Líq Dív.Brut/ Patrim. Cresc. Rec.5a'

## 3. Tratamento dos Dados

### 3.1 Editando o cabeçalho da tabela

In [7]:
chaves =rows[0].text

chaves=chaves.split(" ")

chaves


['Papel',
 'Cotação',
 'P/L',
 'P/VP',
 'PSR',
 'Div.Yield',
 'P/Ativo',
 'P/Cap.Giro',
 'P/EBIT',
 'P/Ativ',
 'Circ.Liq',
 'EV/EBIT',
 'EV/EBITDA',
 'Mrg',
 'Ebit',
 'Mrg.',
 'Líq.',
 'Liq.',
 'Corr.',
 'ROIC',
 'ROE',
 'Liq.2meses',
 'Patrim.',
 'Líq',
 'Dív.Brut/',
 'Patrim.',
 'Cresc.',
 'Rec.5a']

In [8]:
chaves[9]= "P/Ativ Circ.Liq"
chaves[13]= "Mrg Ebit"
chaves[15]= "Mrg Líq."
chaves[17]= "Líq. Corr."
chaves[22]= "Patrim. Líq"
chaves[24]= "Dív.Brut/Patrim."
chaves[26]= "Cresc. Rec.5a"

chaves.remove("Circ.Liq")
chaves.remove("Ebit")
chaves.remove("Líq.")
chaves.remove("Corr.")
chaves.remove("Líq")
chaves.remove("Patrim.")
chaves.remove("Rec.5a")

chaves

['Papel',
 'Cotação',
 'P/L',
 'P/VP',
 'PSR',
 'Div.Yield',
 'P/Ativo',
 'P/Cap.Giro',
 'P/EBIT',
 'P/Ativ Circ.Liq',
 'EV/EBIT',
 'EV/EBITDA',
 'Mrg Ebit',
 'Mrg Líq.',
 'Líq. Corr.',
 'ROIC',
 'ROE',
 'Liq.2meses',
 'Patrim. Líq',
 'Dív.Brut/Patrim.',
 'Cresc. Rec.5a']

### 3.2 Criando um loop para iterar sobre cada linha da tabela e armazenar os dados em uma lista

In [9]:
fila=[]

for i in range(1, len(rows)):
    papel=rows[i].text
    papel= papel.split(" ")
    fila.append(papel)

### 3.3 Criando um dicionário com as listas 'chaves' e 'fila' e armazenando-o à lista 'acoes_list'

In [10]:
acoes_list = []

for i in range(0, len(fila)):
    papel_tuplas = zip(chaves,fila[i])
    papel_dict =dict(papel_tuplas)
    acoes_list.append(papel_dict)

### 3.4 Convertendo a Lista "acoes_list" em um DataFrame

In [11]:
import pandas as pd

papel_df = pd.DataFrame(acoes_list)
papel_df

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,EV/EBITDA,Mrg Ebit,Mrg Líq.,Líq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/Patrim.,Cresc. Rec.5a
0,OIBR3,"0,64","0,02","-0,01","0,127","0,00%","0,011","-0,09","-0,08","-0,01",...,"-4,86","-169,04%","847,43%","0,70","-18,21%","-95,87%","5.349.740,00","-14.660.000.000,00","-0,77","-25,54%"
1,AZUL4,"0,99","0,08","-0,03","0,045","0,00%","0,036","-0,08","0,20","-0,02",...,"4,68","22,26%","58,76%","0,35","22,22%","-41,81%","138.764.000,00","-28.451.100.000,00","-1,22","40,42%"
2,AMER3,"5,68","0,16","0,25","0,083","0,00%","0,065","0,48","-0,87","-0,20",...,"-7,10","-9,54%","53,64%","1,49","-9,43%","163,84%","15.569.900,00","4.475.000.000,00","0,41","-1,96%"
3,LIGT3,"6,09","0,94","0,40","0,148","0,00%","0,085","0,88","1,17","-0,18",...,"3,05","12,63%","15,82%","1,44","9,21%","42,92%","4.673.900,00","5.639.690.000,00","1,77","1,54%"
4,SYNE3,"5,80","1,59","0,81","0,666","120,86%","0,279","0,91","1,89","-7,43",...,"2,35","35,33%","50,48%","5,43","17,26%","51,13%","2.785.690,00","1.091.940.000,00","0,77","3,90%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,ORVR3,"53,14","151,83","5,71","4,714","0,00%","1,301","10,38","17,76","-2,70",...,"14,15","26,54%","4,36%","1,93","8,66%","3,76%","29.428.800,00","771.834.000,00","2,50","26,32%"
169,QUAL3,"2,19","165,12","0,48","0,402","0,00%","0,141","4,51","1,19","-0,47",...,"1,66","33,88%","0,80%","1,08","18,47%","0,29%","4.753.500,00","1.297.730.000,00","1,49","-7,32%"
170,POSI3,"4,58","222,11","0,40","0,191","5,98%","0,154","0,46","2,88","3,89",...,"4,91","6,62%","0,24%","2,03","6,99%","0,18%","7.452.260,00","1.607.650.000,00","0,77","7,83%"
171,SRNA3,"11,49","1.167,43","1,31","1,558","0,00%","0,346","15,13","6,08","-0,61",...,"9,07","25,62%","0,14%","1,16","6,17%","0,11%","86.227.100,00","5.443.630.000,00","2,20","35,93%"


### 3.5 Selecionando Variáveis para Análise

In [12]:
acoes_df =papel_df[["Papel", "Cotação","P/L", "P/VP", "Div.Yield", "EV/EBIT", "Mrg Líq.", "ROE"]]
acoes_df

,Papel,Cotação,P/L,P/VP,Div.Yield,EV/EBIT,Mrg Líq.,ROE
0,OIBR3,"0,64","0,02","-0,01","0,00%","-3,59","847,43%","-95,87%"
1,AZUL4,"0,99","0,08","-0,03","0,00%","7,55","58,76%","-41,81%"
2,AMER3,"5,68","0,16","0,25","0,00%","-1,61","53,64%","163,84%"
3,LIGT3,"6,09","0,94","0,40","0,00%","4,48","15,82%","42,92%"
4,SYNE3,"5,80","1,59","0,81","120,86%","2,74","50,48%","51,13%"
...,...,...,...,...,...,...,...,...
168,ORVR3,"53,14","151,83","5,71","0,00%","23,88","4,36%","3,76%"
169,QUAL3,"2,19","165,12","0,48","0,00%","2,85","0,80%","0,29%"
170,POSI3,"4,58","222,11","0,40","5,98%","6,30","0,24%","0,18%"
171,SRNA3,"11,49","1.167,43","1,31","0,00%","15,20","0,14%","0,11%"


### Metadados:

>  - Papel: ticker do ativo
>  - Cotação: valor atual do ativo em reais
>  - P/L: preço sobre o lucro. Indica quanto o mercado está disposto a pagar pelos lucros gerados pela empresa
>  - P/VP: Preço sobre Valor Patrimonial, Um valor abaixo de 1 pode sugerir que a empresa está sendo negociada por menos do que seu valor patrimonial, indicando uma possível subvalorização.
>  - Div.Yield: Dividend Yield, é o indicador que avalia o desempenho da empresa com base nos proventos pagos aos acionistas nos últimos 12 meses.
>  - EV/EBIT: valor da empresa (Enterprise Value) com o seu lucro antes de juros e impostos (EBIT) 
>  - Mrg Líq.: Margem Líquida, indica quanto da receita é  convertida em lucro
>  - ROE: Retorno sobre o Patrimônio Líquido, é um indicador de rentabilidade que avalia a eficiência de uma empresa em gerar lucro a partir dos seus recursos próprios

In [13]:
# Obter uma descrição dos dados

acoes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Papel      173 non-null    object
 1   Cotação    173 non-null    object
 2   P/L        173 non-null    object
 3   P/VP       173 non-null    object
 4   Div.Yield  173 non-null    object
 5   EV/EBIT    173 non-null    object
 6   Mrg Líq.   173 non-null    object
 7   ROE        173 non-null    object
dtypes: object(8)
memory usage: 10.9+ KB


### 3.6 Convertendo Variáveis de String para Float (Exceto a Variável "Papel")

In [14]:
   
acoes_df1 = acoes_df.copy()

acoes_df1["Div.Yield"]=acoes_df1["Div.Yield"].str.slice(0,4)
acoes_df1 = acoes_df1.rename(columns={'Div.Yield': 'Div.Yield %'})

acoes_df1["Mrg Líq."]=acoes_df1["Mrg Líq."].str.slice(0,4)
acoes_df1 = acoes_df1.rename(columns={'Mrg Líq.': 'Mrg Líq. %'})

acoes_df1["ROE"]=acoes_df1["ROE"].str.slice(0,4)
acoes_df1 = acoes_df1.rename(columns={'ROE': 'ROE %'})

acoes_df1

,Papel,Cotação,P/L,P/VP,Div.Yield %,EV/EBIT,Mrg Líq. %,ROE %
0,OIBR3,"0,64","0,02","-0,01","0,00","-3,59","847,","-95,"
1,AZUL4,"0,99","0,08","-0,03","0,00","7,55","58,7","-41,"
2,AMER3,"5,68","0,16","0,25","0,00","-1,61","53,6","163,"
3,LIGT3,"6,09","0,94","0,40","0,00","4,48","15,8","42,9"
4,SYNE3,"5,80","1,59","0,81","120,","2,74","50,4","51,1"
...,...,...,...,...,...,...,...,...
168,ORVR3,"53,14","151,83","5,71","0,00","23,88","4,36","3,76"
169,QUAL3,"2,19","165,12","0,48","0,00","2,85","0,80","0,29"
170,POSI3,"4,58","222,11","0,40","5,98","6,30","0,24","0,18"
171,SRNA3,"11,49","1.167,43","1,31","0,00","15,20","0,14","0,11"


In [15]:
def aux(x):
    return float(x.replace(".","").replace(",", "."))

acoes_df1['Cotação'] = acoes_df1['Cotação'].apply(aux)
acoes_df1['P/L'] = acoes_df1['P/L'].apply(aux)
acoes_df1['P/VP'] = acoes_df1['P/VP'].apply(aux)
acoes_df1['Div.Yield %'] = acoes_df1['Div.Yield %'].apply(aux)
acoes_df1['EV/EBIT'] = acoes_df1['EV/EBIT'].apply(aux)

acoes_df1['Mrg Líq. %'] = acoes_df1['Mrg Líq. %'].apply(aux)
acoes_df1['ROE %'] = acoes_df1['ROE %'].apply(aux)

acoes_df1

,Papel,Cotação,P/L,P/VP,Div.Yield %,EV/EBIT,Mrg Líq. %,ROE %
0,OIBR3,0.64,0.02,-0.01,0.00,-3.59,847.00,-95.00
1,AZUL4,0.99,0.08,-0.03,0.00,7.55,58.70,-41.00
2,AMER3,5.68,0.16,0.25,0.00,-1.61,53.60,163.00
3,LIGT3,6.09,0.94,0.40,0.00,4.48,15.80,42.90
4,SYNE3,5.80,1.59,0.81,120.00,2.74,50.40,51.10
...,...,...,...,...,...,...,...,...
168,ORVR3,53.14,151.83,5.71,0.00,23.88,4.36,3.76
169,QUAL3,2.19,165.12,0.48,0.00,2.85,0.80,0.29
170,POSI3,4.58,222.11,0.40,5.98,6.30,0.24,0.18
171,SRNA3,11.49,1167.43,1.31,0.00,15.20,0.14,0.11


In [16]:
acoes_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Papel        173 non-null    object 
 1   Cotação      173 non-null    float64
 2   P/L          173 non-null    float64
 3   P/VP         173 non-null    float64
 4   Div.Yield %  173 non-null    float64
 5   EV/EBIT      173 non-null    float64
 6   Mrg Líq. %   173 non-null    float64
 7   ROE %        173 non-null    float64
dtypes: float64(7), object(1)
memory usage: 10.9+ KB


### 3.7 Selecionando um Ticker para Cada Empresa Negociada na B3 (menor EV/EBIT por exemplo)


In [17]:
ticker = []

for indice, linha in acoes_df1.iterrows():
    ticker.append(linha["Papel"][0:4])
    
acoes_df1["Empresa"] = ticker

acoes_df1

,Papel,Cotação,P/L,P/VP,Div.Yield %,EV/EBIT,Mrg Líq. %,ROE %,Empresa
0,OIBR3,0.64,0.02,-0.01,0.00,-3.59,847.00,-95.00,OIBR
1,AZUL4,0.99,0.08,-0.03,0.00,7.55,58.70,-41.00,AZUL
2,AMER3,5.68,0.16,0.25,0.00,-1.61,53.60,163.00,AMER
3,LIGT3,6.09,0.94,0.40,0.00,4.48,15.80,42.90,LIGT
4,SYNE3,5.80,1.59,0.81,120.00,2.74,50.40,51.10,SYNE
...,...,...,...,...,...,...,...,...,...
168,ORVR3,53.14,151.83,5.71,0.00,23.88,4.36,3.76,ORVR
169,QUAL3,2.19,165.12,0.48,0.00,2.85,0.80,0.29,QUAL
170,POSI3,4.58,222.11,0.40,5.98,6.30,0.24,0.18,POSI
171,SRNA3,11.49,1167.43,1.31,0.00,15.20,0.14,0.11,SRNA


In [18]:
acoes_df1 = acoes_df1.sort_values(by="EV/EBIT")
acoes_df1 = acoes_df1.drop_duplicates("Empresa")
acoes_df1 = acoes_df1.drop('Empresa', axis=1)
acoes_df1

,Papel,Cotação,P/L,P/VP,Div.Yield %,EV/EBIT,Mrg Líq. %,ROE %
99,CXSE3,14.60,11.06,3.12,7.85,-332.55,0.00,28.20
20,BRAP4,15.65,5.27,0.71,12.90,-183.09,0.00,13.40
0,OIBR3,0.64,0.02,-0.01,0.00,-3.59,847.00,-95.00
2,AMER3,5.68,0.16,0.25,0.00,-1.61,53.60,163.00
49,SANB3,14.15,7.66,1.18,5.42,0.00,0.00,15.30
...,...,...,...,...,...,...,...,...
168,ORVR3,53.14,151.83,5.71,0.00,23.88,4.36,3.76
18,LOGG3,21.58,5.06,0.52,9.05,23.92,169.00,10.20
146,HYPE3,25.68,20.11,1.39,3.79,24.94,11.90,6.89
167,LAND3,11.52,104.42,1.52,0.63,31.97,14.30,1.46


## 4. Fórmula de Benjamin Graham

Benjamin Graham, estabeleceu dois critérios fundamentais para avaliar o preço justo de uma ação:

1. Relação Preço/Lucro (P/L): O preço de uma ação não deve exceder 15 vezes o lucro médio gerado nos últimos três anos.

2. Relação Preço/Valor Patrimonial (P/VP):O preço por ação, dividido pelo valor patrimonial por ação, não deve ser maior que 1.5.

Graham também propôs que o produto dessas duas relações (P/L e P/VP) não deve exceder 22.5.

A fórmula de Graham, leva em consideração métricas como Lucro por Ação (LPA) e Valor Patrimonial por Ação (VPA).

$$preço\ justo = \sqrt{22.5*LPA*VPA}$$

- Por levar em conta o valor patrimonial em seu cálculo, a fórmula de Benjamin Graham apresenta limitações quando aplicada a empresas de tecnologia.

- Para bancos e seguradoras, pode-se utilizar a fórmula adaptada de Joel Greenblatt.








### 4.1 Aplicando os Critérios de Benjamin Graham ao DataFrame

- P/L <= 15

- P/PV > 0 e P/PV <=1.5

- EV/EBIT <= 10


In [19]:
acoes_df2 = acoes_df1[(acoes_df1['P/L'] <= 15) & (acoes_df1['P/VP'] > 0) & (acoes_df1['P/VP'] <= 1.5) & (acoes_df1['EV/EBIT'] > 0) &  (acoes_df1['EV/EBIT'] <= 10)]
acoes_df2

,Papel,Cotação,P/L,P/VP,Div.Yield %,EV/EBIT,Mrg Líq. %,ROE %
57,GOAU4,8.93,7.95,0.46,3.92,2.53,4.86,5.74
4,SYNE3,5.80,1.59,0.81,120.00,2.74,50.40,51.10
60,PETR4,30.18,8.08,0.98,22.00,3.40,9.78,12.10
8,VTRU3,9.58,3.89,0.49,0.23,4.24,15.10,12.60
47,VALE3,52.53,7.58,1.18,9.05,4.32,14.30,15.50
3,LIGT3,6.09,0.94,0.40,0.00,4.48,15.80,42.90
93,CSED3,5.34,10.13,1.25,7.04,4.95,7.30,12.30
52,RECV3,14.81,7.83,0.97,24.20,5.08,16.40,12.40
23,COGN3,3.01,5.74,0.45,2.22,5.35,16.70,7.90
45,EUCA4,18.99,7.31,0.66,3.87,5.40,8.13,9.07


### 4.2 Extraindo LPA e VPA de Cada Ativo do Site "Fundamentus"

In [20]:
lista_ativos = list(acoes_df2["Papel"])
lista_ativos

['GOAU4',
 'SYNE3',
 'PETR4',
 'VTRU3',
 'VALE3',
 'LIGT3',
 'CSED3',
 'RECV3',
 'COGN3',
 'EUCA4',
 'VLID3',
 'MYPK3',
 'PFRM3',
 'ISAE4',
 'UGPA3',
 'CMIG4',
 'GUAR3',
 'MDNE3',
 'GGBR4',
 'HBRE3',
 'FIQE3',
 'DESK3',
 'CSMG3',
 'MOVI3',
 'ECOR3',
 'ALUP11',
 'ALOS3',
 'RAPT4',
 'NEOE3',
 'GRND3',
 'BLAU3',
 'ARML3',
 'JSLG3',
 'RANI3',
 'SAPR4',
 'CRFB3',
 'ENGI11',
 'CAML3',
 'SBFG3',
 'CYRE3',
 'FLRY3',
 'AGRO3',
 'HBOR3',
 'PRIO3',
 'MDIA3',
 'BMOB3',
 'MTRE3',
 'FESA4',
 'VBBR3',
 'ELET3',
 'TRIS3',
 'SHUL4']

In [21]:
import time

VPA_list = []
LPA_list = []

for papel in lista_ativos:
    navegador.find_element(By.XPATH, '//*[@id="completar"]').send_keys(papel)    
    navegador.find_element(By.XPATH, '/html/body/div[1]/div[1]/form/fieldset/input[2]').click()
    
    time.sleep(1)
    
    VPA = navegador.find_element(By.XPATH, '/html/body/div[1]/div[2]/table[3]/tbody/tr[3]/td[6]/span').text
    VPA = float(VPA.replace(",","."))
    VPA_list.append(VPA)
    
    LPA = navegador.find_element(By.XPATH, '/html/body/div[1]/div[2]/table[3]/tbody/tr[2]/td[6]/span').text
    LPA = float(LPA.replace(",","."))         
    LPA_list.append(LPA)

### 4.3 Adicionando as Colunas LPA e VPA ao DataFrame

In [22]:
acoes_df3 = acoes_df2.copy()
acoes_df3["VPA"]= VPA_list
acoes_df3["LPA"]= LPA_list

acoes_df3

,Papel,Cotação,P/L,P/VP,Div.Yield %,EV/EBIT,Mrg Líq. %,ROE %,VPA,LPA
57,GOAU4,8.93,7.95,0.46,3.92,2.53,4.86,5.74,19.58,1.12
4,SYNE3,5.80,1.59,0.81,120.00,2.74,50.40,51.10,7.15,3.66
60,PETR4,30.18,8.08,0.98,22.00,3.40,9.78,12.10,30.71,3.73
8,VTRU3,9.58,3.89,0.49,0.23,4.24,15.10,12.60,19.44,2.46
47,VALE3,52.53,7.58,1.18,9.05,4.32,14.30,15.50,44.52,6.93
3,LIGT3,6.09,0.94,0.40,0.00,4.48,15.80,42.90,15.14,6.50
93,CSED3,5.34,10.13,1.25,7.04,4.95,7.30,12.30,4.27,0.53
52,RECV3,14.81,7.83,0.97,24.20,5.08,16.40,12.40,15.21,1.89
23,COGN3,3.01,5.74,0.45,2.22,5.35,16.70,7.90,6.63,0.52
45,EUCA4,18.99,7.31,0.66,3.87,5.40,8.13,9.07,28.64,2.60


### 4.4Cálculo do Valor Justo

In [23]:
acoes_df3['Preco_Justo'] = round((22.5*acoes_df3['VPA']*acoes_df3['LPA'])**0.5, 2)

acoes_df3

,Papel,Cotação,P/L,P/VP,Div.Yield %,EV/EBIT,Mrg Líq. %,ROE %,VPA,LPA,Preco_Justo
57,GOAU4,8.93,7.95,0.46,3.92,2.53,4.86,5.74,19.58,1.12,22.21
4,SYNE3,5.80,1.59,0.81,120.00,2.74,50.40,51.10,7.15,3.66,24.27
60,PETR4,30.18,8.08,0.98,22.00,3.40,9.78,12.10,30.71,3.73,50.77
8,VTRU3,9.58,3.89,0.49,0.23,4.24,15.10,12.60,19.44,2.46,32.80
47,VALE3,52.53,7.58,1.18,9.05,4.32,14.30,15.50,44.52,6.93,83.32
3,LIGT3,6.09,0.94,0.40,0.00,4.48,15.80,42.90,15.14,6.50,47.06
93,CSED3,5.34,10.13,1.25,7.04,4.95,7.30,12.30,4.27,0.53,7.14
52,RECV3,14.81,7.83,0.97,24.20,5.08,16.40,12.40,15.21,1.89,25.43
23,COGN3,3.01,5.74,0.45,2.22,5.35,16.70,7.90,6.63,0.52,8.81
45,EUCA4,18.99,7.31,0.66,3.87,5.40,8.13,9.07,28.64,2.60,40.93


### 4.5 Margens de Segurança

In [24]:
acoes_df3['Potencial rentabilidad %'] = round((acoes_df3['Preco_Justo']/acoes_df3['Cotação']-1)*100, 2)
acoes_df3['Margem seguranca 60%'] = round(acoes_df3['Preco_Justo']*0.4, 2)
acoes_df3['Margem seguranca 50%'] = round(acoes_df3['Preco_Justo']*0.5, 2)
acoes_df3['Margem seguranca 40%'] = round(acoes_df3['Preco_Justo']*0.6, 2)

acoes_df3

,Papel,Cotação,P/L,P/VP,Div.Yield %,EV/EBIT,Mrg Líq. %,ROE %,VPA,LPA,Preco_Justo,Potencial rentabilidad %,Margem seguranca 60%,Margem seguranca 50%,Margem seguranca 40%
57,GOAU4,8.93,7.95,0.46,3.92,2.53,4.86,5.74,19.58,1.12,22.21,148.71,8.88,11.10,13.33
4,SYNE3,5.80,1.59,0.81,120.00,2.74,50.40,51.10,7.15,3.66,24.27,318.45,9.71,12.14,14.56
60,PETR4,30.18,8.08,0.98,22.00,3.40,9.78,12.10,30.71,3.73,50.77,68.22,20.31,25.38,30.46
8,VTRU3,9.58,3.89,0.49,0.23,4.24,15.10,12.60,19.44,2.46,32.80,242.38,13.12,16.40,19.68
47,VALE3,52.53,7.58,1.18,9.05,4.32,14.30,15.50,44.52,6.93,83.32,58.61,33.33,41.66,49.99
3,LIGT3,6.09,0.94,0.40,0.00,4.48,15.80,42.90,15.14,6.50,47.06,672.74,18.82,23.53,28.24
93,CSED3,5.34,10.13,1.25,7.04,4.95,7.30,12.30,4.27,0.53,7.14,33.71,2.86,3.57,4.28
52,RECV3,14.81,7.83,0.97,24.20,5.08,16.40,12.40,15.21,1.89,25.43,71.71,10.17,12.72,15.26
23,COGN3,3.01,5.74,0.45,2.22,5.35,16.70,7.90,6.63,0.52,8.81,192.69,3.52,4.40,5.29
45,EUCA4,18.99,7.31,0.66,3.87,5.40,8.13,9.07,28.64,2.60,40.93,115.53,16.37,20.46,24.56


#### 4.5.1 Ativos com Margem de Segurança de Pelo Menos 60%

In [25]:
acoes60 = acoes_df3[acoes_df3['Cotação'] < acoes_df3['Margem seguranca 60%']]
acoes60

,Papel,Cotação,P/L,P/VP,Div.Yield %,EV/EBIT,Mrg Líq. %,ROE %,VPA,LPA,Preco_Justo,Potencial rentabilidad %,Margem seguranca 60%,Margem seguranca 50%,Margem seguranca 40%
4,SYNE3,5.80,1.59,0.81,120.00,2.74,50.4,51.10,7.15,3.66,24.27,318.45,9.71,12.14,14.56
8,VTRU3,9.58,3.89,0.49,0.23,4.24,15.1,12.60,19.44,2.46,32.80,242.38,13.12,16.40,19.68
3,LIGT3,6.09,0.94,0.40,0.00,4.48,15.8,42.90,15.14,6.50,47.06,672.74,18.82,23.53,28.24
23,COGN3,3.01,5.74,0.45,2.22,5.35,16.7,7.90,6.63,0.52,8.81,192.69,3.52,4.40,5.29
13,ISAE4,23.27,4.30,0.75,10.10,5.76,43.1,17.40,30.98,5.42,61.47,164.16,24.59,30.74,36.88
95,HBRE3,3.57,10.39,0.18,0.00,6.28,70.9,1.74,19.78,0.34,12.30,244.54,4.92,6.15,7.38
10,SAPR4,6.59,4.19,0.83,4.79,7.87,34.1,19.70,7.96,1.57,16.77,154.48,6.71,8.38,10.06
29,HBOR3,2.68,6.28,0.25,3.77,8.78,12.9,3.91,10.91,0.43,10.27,283.21,4.11,5.14,6.16
40,MTRE3,4.03,7.18,0.43,11.20,9.43,7.1,5.95,9.43,0.56,10.90,170.47,4.36,5.45,6.54


#### 4.5.2 Ativos com Margem de Segurança de Pelo Menos 50%

In [26]:
acoes50 = acoes_df3[(acoes_df3['Cotação'] <= acoes_df3['Margem seguranca 50%']) & (acoes_df3['Cotação'] > acoes_df3['Margem seguranca 60%'])]
acoes50

,Papel,Cotação,P/L,P/VP,Div.Yield %,EV/EBIT,Mrg Líq. %,ROE %,VPA,LPA,Preco_Justo,Potencial rentabilidad %,Margem seguranca 60%,Margem seguranca 50%,Margem seguranca 40%
57,GOAU4,8.93,7.95,0.46,3.92,2.53,4.86,5.74,19.58,1.12,22.21,148.71,8.88,11.10,13.33
45,EUCA4,18.99,7.31,0.66,3.87,5.40,8.13,9.07,28.64,2.60,40.93,115.53,16.37,20.46,24.56
71,MYPK3,12.60,8.59,0.45,5.65,5.63,2.13,5.23,28.02,1.47,30.44,141.59,12.18,15.22,18.26
14,CMIG4,10.68,4.36,1.09,11.90,5.89,17.20,25.00,9.75,2.45,23.18,117.04,9.27,11.59,13.91
53,CAML3,4.88,7.87,0.49,4.51,7.99,1.77,6.28,9.88,0.62,11.74,140.57,4.70,5.87,7.04
5,PRIO3,40.95,3.24,1.38,0.00,8.91,72.60,42.50,29.68,12.64,91.87,124.35,36.75,45.94,55.12
7,VBBR3,20.45,3.70,1.11,8.72,9.62,3.48,30.10,18.37,5.53,47.81,133.79,19.12,23.90,28.69


#### 4.5.3 Ativos com Margem de Segurança de Pelo Menos 40%

In [27]:
acoes40 = acoes_df3[(acoes_df3['Cotação'] <= acoes_df3['Margem seguranca 40%']) & (acoes_df3['Cotação'] > acoes_df3['Margem seguranca 50%'])]
acoes40

,Papel,Cotação,P/L,P/VP,Div.Yield %,EV/EBIT,Mrg Líq. %,ROE %,VPA,LPA,Preco_Justo,Potencial rentabilidad %,Margem seguranca 60%,Margem seguranca 50%,Margem seguranca 40%
60,PETR4,30.18,8.08,0.98,22.00,3.40,9.78,12.10,30.71,3.73,50.77,68.22,20.31,25.38,30.46
52,RECV3,14.81,7.83,0.97,24.20,5.08,16.40,12.40,15.21,1.89,25.43,71.71,10.17,12.72,15.26
67,PFRM3,8.52,8.47,0.80,6.75,5.64,1.25,9.47,10.62,1.01,15.54,82.39,6.22,7.77,9.32
33,MDNE3,21.56,6.55,1.14,5.76,6.18,16.40,17.30,18.98,3.29,37.48,73.84,14.99,18.74,22.49
92,GGBR4,16.24,10.07,0.59,4.07,6.22,4.85,5.84,27.62,1.61,31.63,94.77,12.65,15.82,18.98
32,CSMG3,23.95,6.53,1.10,6.76,6.39,17.20,16.80,21.82,3.67,42.45,77.24,16.98,21.22,25.47
76,NEOE3,25.36,8.77,0.91,3.06,7.03,7.13,10.30,27.85,2.89,42.56,67.82,17.02,21.28,25.54
64,JSLG3,5.95,8.29,0.95,7.07,7.79,2.21,11.40,6.25,0.72,10.06,69.08,4.02,5.03,6.04
25,RANI3,7.83,5.82,1.33,8.46,7.83,19.30,22.80,5.89,1.35,13.38,70.88,5.35,6.69,8.03
85,CRFB3,8.48,9.22,0.85,1.12,7.91,1.82,9.21,9.98,0.92,14.37,69.46,5.75,7.18,8.62


### 5. Ranking dos Ativos

- Para realizar o ranking, os indicadores a serem considerados são: Potencial de Rentabilidade, Dividend Yield, Margem Líquida e ROE. Todos esses indicadores têm melhor desempenho quanto maiores forem os seus valores.

In [28]:
Rank_df = acoes_df3[acoes_df3['Cotação'] <= acoes_df3['Margem seguranca 40%']]

Ranking1 =Rank_df[["Papel", "Cotação","Potencial rentabilidad %","EV/EBIT", "Div.Yield %", "Mrg Líq. %", "ROE %"]]

Ranking = Ranking1.copy()

Ranking['PR_rank'] = Ranking['Potencial rentabilidad %'].rank(ascending=False)

Ranking['EV/EBIT_rank'] = Ranking['EV/EBIT'].rank()

Ranking['DY_rank'] = Ranking['Div.Yield %'].rank(ascending=False)

Ranking['ML_rank'] = Ranking['Mrg Líq. %'].rank(ascending=False)

Ranking['ROE_rank'] = Ranking['ROE %'].rank(ascending=False)

Ranking['suma_rank'] = Ranking['PR_rank'] + Ranking['DY_rank'] + Ranking['ML_rank'] + Ranking['ROE_rank']
Ranking = Ranking.sort_values(by="suma_rank")

Ranking



,Papel,Cotação,Potencial rentabilidad %,EV/EBIT,Div.Yield %,Mrg Líq. %,ROE %,PR_rank,EV/EBIT_rank,DY_rank,ML_rank,ROE_rank,suma_rank
4,SYNE3,5.80,318.45,2.74,120.00,50.40,51.10,2.0,2.0,1.0,3.0,1.0,7.0
13,ISAE4,23.27,164.16,5.76,10.10,43.10,17.40,8.0,11.0,6.0,4.0,11.0,29.0
14,CMIG4,10.68,117.04,5.89,11.90,17.20,25.00,15.0,12.0,4.0,10.5,5.0,34.5
10,SAPR4,6.59,154.48,7.87,4.79,34.10,19.70,9.0,20.0,17.0,5.0,8.0,39.0
25,RANI3,7.83,70.88,7.83,8.46,19.30,22.80,27.0,19.0,8.0,8.0,6.0,49.0
3,LIGT3,6.09,672.74,4.48,0.00,15.80,42.90,1.0,5.0,32.0,15.0,2.0,50.0
5,PRIO3,40.95,124.35,8.91,0.00,72.60,42.50,14.0,28.0,32.0,1.0,3.0,50.0
7,VBBR3,20.45,133.79,9.62,8.72,3.48,30.10,13.0,31.0,7.0,28.0,4.0,52.0
39,AGRO3,21.57,75.38,8.55,7.21,22.80,14.20,23.0,26.0,10.0,7.0,14.0,54.0
32,CSMG3,23.95,77.24,6.39,6.76,17.20,16.80,22.0,16.0,12.0,10.5,13.0,57.5


## 5. Conclusão

- Antes de montar uma carteira de ativos, é fundamental analisar cada empresa por meio de seus relatórios financeiros e de desempenho. Isso permite construir uma carteira diversificada, selecionando os ativos com base no princípio da margem de segurança ou utilizando um ranking que combina diferentes indicadores financeiros.